### Use a pre-trained convolution neural network (VGG16) for image classification

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()


In [3]:
# Normalize pixel values to be between 0 and 1
x_train = x_train / 255.0
x_test = x_test / 255.0


In [4]:
# One-hot encode labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [5]:
# Load the VGG16 model pre-trained on ImageNet, without the top layers (for transfer learning)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))


In [6]:
# Add custom classification layers on top of the VGG16 model
x = base_model.output
x = Flatten()(x)  # Flatten the output of the base model
predictions = Dense(10, activation='softmax')(x)  # Add a dense layer with 10 outputs (one for each CIFAR-10 class)

In [7]:
# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

In [8]:
# Freeze the base model layers (we'll only train the custom dense layers)
for layer in base_model.layers:
    layer.trainable = False

In [9]:
# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
# Use ImageDataGenerator to apply data augmentation for training
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [11]:
# Fit the model on the training data using data augmentation
model.fit(datagen.flow(x_train, y_train, batch_size=32),
          epochs=4,
          validation_data=(x_test, y_test))

Epoch 1/4


C:\Users\akanksh_02\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1563/1563 ━━━━━━━━━━━━━━━━━━━━ 121s 77ms/step - accuracy: 0.3374 - loss: 1.8730 - val_accuracy: 0.4972 - val_loss: 1.4663
Epoch 2/4
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 120s 76ms/step - accuracy: 0.4590 - loss: 1.5529 - val_accuracy: 0.5246 - val_loss: 1.3787
Epoch 3/4
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 116s 74ms/step - accuracy: 0.4816 - loss: 1.4843 - val_accuracy: 0.5302 - val_loss: 1.3525
Epoch 4/4
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 121s 77ms/step - accuracy: 0.4938 - loss: 1.4636 - val_accuracy: 0.5339 - val_loss: 1.3450


In [12]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(x_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 63ms/step - accuracy: 0.5336 - loss: 1.3456


In [13]:
# Print the results
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

Test Loss: 1.3449794054031372
Test Accuracy: 0.5339000225067139
